In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np

import sys
sys.path.append("./missing_data")

from mpl_toolkits.basemap import Basemap
from build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile, calc_tile_offsets, save_single_tile
from build_dataset import split_into_tiles_subfolders_and_track_dates, create_tile_videos, create_and_save_tile_from_complete_df
from medicane_utils.geo_const import latcorners, loncorners, x_center, y_center, basemap_obj
from medicane_utils.load_files import load_all_images

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Xmin, Ymin, px_scale_x, px_scale_y = compute_pixel_scale()
1/px_scale_y/1000 * 224, 1/px_scale_x/1000 * 224

(672.3640767222608, 672.0122685182674)

In [2]:
# Percorso alla cartella dei frame
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/" 

#### load all file names and dates

In [ ]:
#os.makedirs(output_dir, exist_ok=True)
sorted_metadata_files = load_all_images(input_dir)

In [ ]:
f_sorted_metadata = sorted_metadata_files[:32]
#f_sorted_metadata

In [ ]:
num_total_files = len(f_sorted_metadata)
# quanti blocchi di 16 consecutivi
num_frames = 16
num_subfolders = num_total_files // num_frames

### Dataframe di tutte le tiles etichettate - check

In [3]:
nome_file = "all_data_wo_overlap_tiles.csv"
df_data = pd.read_csv(nome_file, dtype={
        "path": 'string',
        "tile_offset_x": 'int16',
        "tile_offset_y": 'int16',
        "label": 'category',
        "lat": 'float32',
        "lon": 'float32',
        "x_pix": 'Int16',
        "y_pix": 'Int16',
        "name": 'string'
    }, parse_dates=['datetime'])
df_data.drop(columns="Unnamed: 0", inplace=True)

In [4]:
df_data

,path,datetime,tile_offset_x,tile_offset_y,label,lat,lon,x_pix,y_pix,name
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,0,NaN,NaN,<NA>,<NA>,[]
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,224,0,0,NaN,NaN,<NA>,<NA>,[]
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,448,0,0,NaN,NaN,<NA>,<NA>,[]
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,672,0,0,NaN,NaN,<NA>,<NA>,[]
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,896,0,0,NaN,NaN,<NA>,<NA>,[]
...,...,...,...,...,...,...,...,...,...,...
1177635,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,0,196,0,NaN,NaN,<NA>,<NA>,[]
1177636,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,224,196,0,NaN,NaN,<NA>,<NA>,[]
1177637,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,448,196,0,NaN,NaN,<NA>,<NA>,[]
1177638,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,672,196,0,NaN,NaN,<NA>,<NA>,[]


In [5]:
# separo gruppi temporali contigui
df_data = df_data.sort_values('datetime') 
# Calcola la differenza temporale rispetto alla riga precedente
df_data['delta'] = df_data['datetime'].diff()

# Definisci i punti di rottura: True se la differenza è maggiore della frequenza attesa
df_data['new_group'] = (df_data['delta'] > pd.Timedelta(minutes=60))  # puoi aumentare il margine se serve

# Crea gli ID di gruppo cumulando i True
df_data['gruppo'] = df_data['new_group'].cumsum()

gruppi_date = [g for _, g in df_data.groupby('gruppo')]

### verifico i gruppi di periodi contigui

In [6]:
lista_date = []
lungh_gruppi = []
date_inizio_fine_gruppi = []
for df in gruppi_date:
    group_datetime = list(df.groupby("datetime"))
    for gd in group_datetime:        
        lista_date.append(len(gd[1]))
    lungh_gruppi.append(len(df)/len(gd[1])) # divido per le 20 tile di ciascun istante temporale
    start_date = df.datetime.iloc[0]
    end_date = df.datetime.iloc[-1]
    date_inizio_fine_gruppi.append((start_date, end_date))


In [7]:
np.array(lungh_gruppi)/288, set(lista_date) # abbiamo 20 tile per ogni data

(array([ 10.00347222,  10.00347222,   8.00347222,   8.00347222,
          4.98611111,   2.93402778,   5.37847222,  54.65625   ,
        277.73611111,   4.00347222,   5.37847222,   8.97916667,
          0.81597222,   8.02083333]),
 {10})

In [8]:
date_inizio_fine_gruppi

[(Timestamp('2011-11-01 00:00:00'), Timestamp('2011-11-11 00:00:00')),
 (Timestamp('2014-11-03 00:00:00'), Timestamp('2014-11-13 00:00:00')),
 (Timestamp('2016-10-26 00:00:00'), Timestamp('2016-11-03 00:00:00')),
 (Timestamp('2017-11-13 00:00:00'), Timestamp('2017-11-21 00:00:00')),
 (Timestamp('2018-09-25 00:00:00'), Timestamp('2018-09-29 23:35:00')),
 (Timestamp('2018-09-30 01:40:00'), Timestamp('2018-10-03 00:00:00')),
 (Timestamp('2020-01-01 00:00:00'), Timestamp('2020-01-06 09:00:00')),
 (Timestamp('2020-02-03 09:05:00'), Timestamp('2020-03-29 00:55:00')),
 (Timestamp('2020-03-29 02:00:00'), Timestamp('2020-12-31 23:55:00')),
 (Timestamp('2021-10-29 00:00:00'), Timestamp('2021-11-02 00:00:00')),
 (Timestamp('2021-11-04 00:00:00'), Timestamp('2021-11-09 09:00:00')),
 (Timestamp('2023-02-24 00:00:00'), Timestamp('2023-03-05 00:00:00')),
 (Timestamp('2023-09-03 00:00:00'), Timestamp('2023-09-03 19:30:00')),
 (Timestamp('2023-09-03 23:25:00'), Timestamp('2023-09-12 00:00:00'))]

### Sembra ben formato -> procedo a creare i video, per ogni blocco temporale

In [43]:
df = gruppi_date[8]

In [10]:
#df

In [ ]:
#grouped = df.groupby(["tile_offset_x", "tile_offset_y"], group_keys=False)  # ottengo 20 gruppi  perché tante sono le tiles

In [44]:
df_videos = create_tile_videos(df)

m_cicloni = df_videos.label == 1
m_non_cicloni = df_videos.label == 0

df_cicloni = df_videos[m_cicloni]
df_non_cicloni = df_videos[m_non_cicloni]

print(len(df_cicloni), len(df_non_cicloni), len(df_videos))

1350 48640 49990


In [ ]:

#for idx, row in df_res[m_cicloni].iterrows():
#    print(row)

138 1662 1800


In [45]:
df_0_balanced = df_non_cicloni.sample(len(df_cicloni))
len(df_0_balanced)

1350

In [46]:
# riduco un pò a piacere i dataframe
df_cicloni = df_cicloni[:500]
df_0_balanced = df_0_balanced[:500]

In [47]:
df_cicloni

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
3105,3105,0,0,17-09-2020_1830_0_0,1,2020-09-17 17:10:00,2020-09-17 18:30:00,"[../fromgcloud/airmass_rgb_20200917_1710.png, ..."
3106,3106,0,0,17-09-2020_1950_0_0,1,2020-09-17 18:30:00,2020-09-17 19:50:00,"[../fromgcloud/airmass_rgb_20200917_1830.png, ..."
3107,3107,0,0,17-09-2020_2110_0_0,1,2020-09-17 19:50:00,2020-09-17 21:10:00,"[../fromgcloud/airmass_rgb_20200917_1950.png, ..."
3108,3108,0,0,17-09-2020_2230_0_0,1,2020-09-17 21:10:00,2020-09-17 22:30:00,"[../fromgcloud/airmass_rgb_20200917_2110.png, ..."
3109,3109,0,0,17-09-2020_2350_0_0,1,2020-09-17 22:30:00,2020-09-17 23:50:00,"[../fromgcloud/airmass_rgb_20200917_2230.png, ..."
...,...,...,...,...,...,...,...,...
24480,24480,448,0,03-12-2020_0925_448_0,1,2020-12-03 08:05:00,2020-12-03 09:25:00,"[../fromgcloud/airmass_rgb_20201203_0805.png, ..."
24481,24481,448,0,03-12-2020_1045_448_0,1,2020-12-03 09:25:00,2020-12-03 10:45:00,"[../fromgcloud/airmass_rgb_20201203_0925.png, ..."
24482,24482,448,0,03-12-2020_1205_448_0,1,2020-12-03 10:45:00,2020-12-03 12:05:00,"[../fromgcloud/airmass_rgb_20201203_1045.png, ..."
24483,24483,448,0,03-12-2020_1325_448_0,1,2020-12-03 12:05:00,2020-12-03 13:25:00,"[../fromgcloud/airmass_rgb_20201203_1205.png, ..."


In [48]:
# creazione delle folder per i video
create_and_save_tile_from_complete_df(df_0_balanced, output_dir)
create_and_save_tile_from_complete_df(df_cicloni, output_dir)

In [49]:
#df_creati_2011 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2014 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2016 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2017 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2020_1 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2020_2 = pd.concat([df_cicloni, df_0_balanced])
df_creati_2020_3 = pd.concat([df_cicloni, df_0_balanced])

In [50]:
print(df_creati_2020_3.shape)
#df_creati_2020_3.to_csv("df_test_2020.csv")

(1000, 8)


In [21]:
df_train = df_creati_2011

In [51]:
df_test = df_creati_2020_3

In [39]:
df_train = pd.concat([df_creati_2011,
df_creati_2014,
df_creati_2016,
df_creati_2017,
#df_creati_2020_1,
#df_creati_2020_2
])

In [53]:
print(df_train.shape)
#df_train.to_csv("df_train.csv")

(854, 8)


### Creazione csv per il training

In [54]:
def create_final_df_csv(df_in, output_dir):
    df_dataset_csv = df_in[['path', 'label']]
    df_dataset_csv['path'] = output_dir + df_dataset_csv['path']
    df_dataset_csv['start'] = 1
    df_dataset_csv['end'] = 16
    df_dataset_csv = df_dataset_csv[['path', 'start', 'end', 'label']]
    return df_dataset_csv


In [55]:
df_train

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
435,435,224,0,05-11-2011_0520_224_0,1,2011-11-05 04:00:00,2011-11-05 05:20:00,"[../fromgcloud/airmass_rgb_20111105_0400.png, ..."
436,436,224,0,05-11-2011_0640_224_0,1,2011-11-05 05:20:00,2011-11-05 06:40:00,"[../fromgcloud/airmass_rgb_20111105_0520.png, ..."
437,437,224,0,05-11-2011_0800_224_0,1,2011-11-05 06:40:00,2011-11-05 08:00:00,"[../fromgcloud/airmass_rgb_20111105_0640.png, ..."
438,438,224,0,05-11-2011_0920_224_0,1,2011-11-05 08:00:00,2011-11-05 09:20:00,"[../fromgcloud/airmass_rgb_20111105_0800.png, ..."
439,439,224,0,05-11-2011_1040_224_0,1,2011-11-05 09:20:00,2011-11-05 10:40:00,"[../fromgcloud/airmass_rgb_20111105_0920.png, ..."
...,...,...,...,...,...,...,...,...
1113,1113,672,196,18-11-2017_2120_672_196,0,2017-11-18 20:00:00,2017-11-18 21:20:00,"[../fromgcloud/airmass_rgb_20171118_2000.png, ..."
910,910,672,0,15-11-2017_1440_672_0,0,2017-11-15 13:20:00,2017-11-15 14:40:00,"[../fromgcloud/airmass_rgb_20171115_1320.png, ..."
1315,1315,896,196,14-11-2017_0240_896_196,0,2017-11-14 01:20:00,2017-11-14 02:40:00,"[../fromgcloud/airmass_rgb_20171114_0120.png, ..."
902,902,672,0,15-11-2017_0400_672_0,0,2017-11-15 02:40:00,2017-11-15 04:00:00,"[../fromgcloud/airmass_rgb_20171115_0240.png, ..."


In [56]:
df_dataset_csv_train = create_final_df_csv(df_train, output_dir)
df_dataset_csv_test = create_final_df_csv(df_test, output_dir)

In [57]:
df_dataset_csv_train

,path,start,end,label
435,../airmassRGB/supervised/05-11-2011_0520_224_0,1,16,1
436,../airmassRGB/supervised/05-11-2011_0640_224_0,1,16,1
437,../airmassRGB/supervised/05-11-2011_0800_224_0,1,16,1
438,../airmassRGB/supervised/05-11-2011_0920_224_0,1,16,1
439,../airmassRGB/supervised/05-11-2011_1040_224_0,1,16,1
...,...,...,...,...
1113,../airmassRGB/supervised/18-11-2017_2120_672_196,1,16,0
910,../airmassRGB/supervised/15-11-2017_1440_672_0,1,16,0
1315,../airmassRGB/supervised/14-11-2017_0240_896_196,1,16,0
902,../airmassRGB/supervised/15-11-2017_0400_672_0,1,16,0


In [60]:
df_dataset_csv_train.to_csv("./train_supervised.csv", index=False)

In [61]:
df_dataset_csv_test.to_csv("./val_supervised.csv", index=False)

In [104]:
df_out[df_out.label ==1]

,path,start,end,label,start_time,end_time
22395,../airmassRGB/supervised/part316_336_0,1,16,1,2014-11-10 11:55:00,2014-11-10 13:10:00
22396,../airmassRGB/supervised/part317_336_0,1,16,1,2014-11-10 13:15:00,2014-11-10 14:30:00
23716,../airmassRGB/supervised/part1637_336_0,1,16,1,2020-03-15 23:05:00,2020-03-16 00:20:00
23717,../airmassRGB/supervised/part1638_336_0,1,16,1,2020-03-16 00:25:00,2020-03-16 01:40:00
23718,../airmassRGB/supervised/part1639_336_0,1,16,1,2020-03-16 01:45:00,2020-03-16 03:00:00
...,...,...,...,...,...,...
139242,../airmassRGB/supervised/part6763_896_112,1,16,1,2020-12-25 23:05:00,2020-12-26 00:20:00
139243,../airmassRGB/supervised/part6764_896_112,1,16,1,2020-12-26 00:25:00,2020-12-26 01:40:00
139244,../airmassRGB/supervised/part6765_896_112,1,16,1,2020-12-26 01:45:00,2020-12-26 03:00:00
139245,../airmassRGB/supervised/part6766_896_112,1,16,1,2020-12-26 03:05:00,2020-12-26 04:20:00


In [84]:
df_out = df_out.rename(columns={'folder':'path'})

In [85]:
df_out.to_csv("./df_videotiles_date.csv", index=False)

### Guardo alcuni video da cancellare dal disco per risparmiare spazio

In [128]:
df_out[df_out['path'].str.contains(r"/part43\d+", regex=True)].label.sum()

0

In [129]:
df_filt = df_out[df_out['path'].apply(os.path.isdir)]
print(f"Numero di cartelle esistenti: {len(df_filt)}, righe totali precedenti: {df_out.shape[0]}")

Numero di cartelle esistenti: 126001, righe totali precedenti: 147200


In [130]:
df_filt[df_filt.label==1].shape[0], df_filt[df_filt.label==0].shape[0], round(df_filt.label.sum()/df_filt.shape[0], 3)

(1512, 124489, 0.012)

In [62]:
def balance_df_by_label(df, label_column="label", random_state=478562):
    """
    Bilancia un DataFrame mantenendo tutte le righe con label=1
    e selezionando (in modo random) lo stesso numero di righe con label=0.
    Restituisce un DataFrame mescolato con uguale quantità di 1 e 0.
    
    Parametri:
    - df: DataFrame originale
    - label_column: il nome della colonna che contiene la label (default: 'label')
    - random_state: per riproducibilità del campionamento casuale
    """
    # Seleziona tutte le righe con label=1
    df_pos = df[df[label_column] == 1]
    # Seleziona tutte le righe con label=0
    df_neg = df[df[label_column] == 0]

    # Numero di righe positive
    num_pos = len(df_pos)
    # Esegui un sample sulle righe negative pari a num_pos
    df_neg_sampled = df_neg.sample(n=num_pos, random_state=random_state)

    # Concatena e mescola
    df_balanced = pd.concat([df_pos, df_neg_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [131]:
df_balanced = balance_df_by_label(df_filt)
print(len(df_balanced), df_balanced['label'].value_counts())

3024 label
1    1512
0    1512
Name: count, dtype: int64


In [137]:
df_balanced


,path,start,end,label,start_time,end_time
0,../airmassRGB/supervised/part6762_784_112,1,16,1,2020-12-25 21:45:00,2020-12-25 23:00:00
1,../airmassRGB/supervised/part6102_784_112,1,16,1,2020-11-19 04:50:00,2020-11-19 06:05:00
2,../airmassRGB/supervised/part6101_784_112,1,16,1,2020-11-19 03:30:00,2020-11-19 04:45:00
3,../airmassRGB/supervised/part3246_560_0,1,16,1,2020-06-13 11:20:00,2020-06-13 12:35:00
4,../airmassRGB/supervised/part1925_336_112,1,16,0,2020-04-01 00:05:00,2020-04-01 01:20:00
...,...,...,...,...,...,...
3019,../airmassRGB/supervised/part1653_560_0,1,16,1,2020-03-16 20:25:00,2020-03-16 21:40:00
3020,../airmassRGB/supervised/part1993_896_112,1,16,0,2020-04-04 18:45:00,2020-04-04 20:00:00
3021,../airmassRGB/supervised/part3207_448_112,1,16,1,2020-06-11 07:20:00,2020-06-11 08:35:00
3022,../airmassRGB/supervised/part2788_336_112,1,16,0,2020-05-18 23:05:00,2020-05-19 00:20:00


In [135]:
l = df_balanced.shape[0]
#l = 100
tr_p = int(l * 0.7)
val_p = int(l * 0.9)
print(f"ranges train e validation:  {tr_p} - {val_p}")
df_train = df_balanced[:tr_p]
df_val = df_balanced[tr_p:val_p]
df_test = df_balanced[val_p:]

ranges train e validation:  2116 - 2721


In [136]:
out_csv_label = os.path.join("./", "train_supervised.csv")
df_train.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "val_supervised.csv")
df_val.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "test_supervised.csv")
df_test.to_csv(out_csv_label, index=False)

### copia le immagini in sottocartelle .../partN

In [27]:
from build_supervised_dataset_no_tiles import split_into_subfolders_and_track_dates

In [ ]:
#subfolders = split_into_subfolders(sorted_filenames, output_dir)
subfolder_info = split_into_subfolders_and_track_dates(sorted_filenames, output_dir)

In [ ]:
#subfolder_info

### Suddivide in train/test/val e scrive i CSV

In [28]:
output_dir

'E:/Medicanes_Data/airmassRGB'

In [30]:
from build_dataset import create_csv

In [33]:
create_csv(output_dir)

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


# Per il dataset supervisionato 
(senza tiles -> sbagliato perché ogni etichetta va associata alla singola tile)

In [ ]:
from build_supervised_dataset_no_tiles import create_supervised_csv_from_info, is_in_medicane
from medicane_utils.load_files import load_medicane_intervals

In [36]:
folder_root = output_dir      # cartella con subfolder part1, part2 ...
medicane_csv = "./medicane_validi.csv"    # le date dei medicane
out_csv = "./dataset.csv"
create_supervised_csv_from_info(subfolder_info, medicane_csv, out_csv)

Creato CSV supervisionato in: ./dataset.csv
